<a href="https://colab.research.google.com/github/guidobarra/pyhton-GPU/blob/main/BarraQuelca_GuidoAlberto_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introducción

El siguiente ejemplo se realizara la **Adicíon Matricial** A y B, el resultado de la Adicíon se guardara en la matriz C.
Hay varias definiciones de Adicíon Matricial, por mencionar algunos: 

**Suma:** denotada por el signo +, suma el componente de la matriz A y el componente de la matriz B y el resultado de esta suma es un componente de la matriz C. Realiza la suma componente a componente de las matrices.

**Suma Directa**: denotada por ⊕, no suma componente a componente como la Suma, sino que ambas matrices conviven en una matriz C de mayor dimension que la matrices A Y B. Cada componente de la diagonal principal de la matriz C es una matriz, el resto de los componentes tiene ceros. Esta difinicion de Adicíon no necesita que sus matrices tengan las mismas dimensiones.

En este ejemplo se realizara la Adicíon Matricial de **Suma**, la cual es la mas conocida y mas utilizada. Como se explico antes la Adicíon Matricial de **Suma** realiza la suma componente a componente de las matrices, ademas de esto las matrices deben tener las mismas dimensiones, si las dimensiones no son igual no se puede realizar la **Suma**


El desarrollo teorico de la **Suma** de matrices se mostrara a continuacion. Realiza la suma componente a componente de las matrices

\begin{aligned}\mathbf {A} +\mathbf {B} &={\begin{bmatrix}a_{11}&a_{12}&\cdots &a_{1n}\\a_{21}&a_{22}&\cdots &a_{2n}\\\vdots &\vdots &\ddots &\vdots \\a_{m1}&a_{m2}&\cdots &a_{mn}\\\end{bmatrix}}+{\begin{bmatrix}b_{11}&b_{12}&\cdots &b_{1n}\\b_{21}&b_{22}&\cdots &b_{2n}\\\vdots &\vdots &\ddots &\vdots \\b_{m1}&b_{m2}&\cdots &b_{mn}\\\end{bmatrix}}\\&={\begin{bmatrix}a_{11}+b_{11}&a_{12}+b_{12}&\cdots &a_{1n}+b_{1n}\\a_{21}+b_{21}&a_{22}+b_{22}&\cdots &a_{2n}+b_{2n}\\\vdots &\vdots &\ddots &\vdots \\a_{m1}+b_{m1}&a_{m2}+b_{m2}&\cdots &a_{mn}+b_{mn}\\\end{bmatrix}}\\\end{aligned}


La condicion para poder realizar la suma de matrices es que la cantidad de filas de la matriz A tiene que ser igual a la cantidad de filas de la matriz B, y ademas cantidad de columnas de la matriz A tiene que ser igual a la cantidad de columnas de la matriz B, en la imagen de arriba se ve como la matriz A y B cumple con esta condion.



EL objetivo es enseñar el funcionamiento del Lenguaje Python, C++, OpenMP y el manejo de la operacion de matriz a bajo nivel. El ejemplo es ilustrativo para entender y utilizar los core que tiene un procesador, realizar proceso en paralelo y sacar concluciones de la biblioteca OpenMP programar un codigo que se va realizar de forma paralela en los core del procesador.

# 2 Armado del ambiente
Se crea un archivo .cpp con el nombre de suma_matriz_axpy, este archivo contiene el codigo, el cual ejecutara la suma de matrices de forma secuencial y tambien la suma de matrices de forma paralela utilizando los core que tiene la maquina.

In [ ]:
# Codigo C++.
code = """
// Axpy con OpenMP, usando C++, ejecutado en Colab. 

#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP   

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_AXPY_SEC      2
#define HTH_AXPY_OMP      3

// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
  TIEMPO_INI( HTH_TOTAL )

  // validar parametros.
  if( argc != 4 )
  {
    std::cout<<argv[1]<<std::endl;
    std::cout<<argv[2]<<std::endl;
    std::cout<<argv[3]<<std::endl;
    std::cerr<< " Error en los parametros, los cantidad de parametros deben ser 3:"<<std::endl<<" (alfa), (beta), (Tamaño de la matriz cantidad_N)."<<argc<<std::endl;
    exit( -1 );
  }

  // Obtener parametros
  float alfa     = atof( argv[1] );
  float beta     = atoi( argv[2] );
  int cantidad_N = atoi( argv[3] );
  
  // Defino la memoria de las Matriz A, B y C. Inicializo con unos la matriz A y con el numero dos la matriz B
  std::vector<std::vector<double>> matriz_A(cantidad_N, std::vector<double> (cantidad_N, 1));
  std::vector<std::vector<double>> matriz_B(cantidad_N, std::vector<double> (cantidad_N, 2));
  std::vector<std::vector<double>> matriz_C_secuencial(cantidad_N, std::vector<double> (cantidad_N, 0));
  std::vector<std::vector<double>> matriz_C_paralela(cantidad_N, std::vector<double> (cantidad_N, 0));

  // Realizo la función Axpy en forma secuencial.

  TIEMPO_INI( HTH_AXPY_SEC )

  for (int i=0;i<cantidad_N;i++)
  {
    for (int j=0;j<cantidad_N;j++)
    {
      matriz_C_secuencial[i][j] = alfa*matriz_A[i][j] + beta*matriz_B[i][j];
    }
  }

  TIEMPO_FIN( HTH_AXPY_SEC )


  // Realizo la función Axpy con OpenMP, paralela.

  TIEMPO_INI( HTH_AXPY_OMP )

  #pragma omp parallel for
  for (int i=0;i<cantidad_N;i++)
  {
    int limit = cantidad_N/2;
    for (int j=0;j<limit;j++)
    {
      matriz_C_paralela[i][j] = alfa*matriz_A[i][j] + beta*matriz_B[i][j];
      matriz_C_paralela[i][cantidad_N-j-1] = alfa*matriz_A[i][cantidad_N-j-1] + beta*matriz_B[i][cantidad_N-j-1];
    }
    if(cantidad_N%2 != 0) {
      matriz_C_paralela[i][limit] = alfa*matriz_A[i][limit] + beta*matriz_B[i][limit];
    }
  }

  TIEMPO_FIN( HTH_AXPY_OMP )

  // --------------------------------------------
  // Muestro los resultados.
  /*
  std::cout<<" matriz paralela :"<<std::endl;
  std::cout<<"["; 
  for(int i=0;i<cantidad_N;i++)
  {
    for(int c=0;c<cantidad_N;c++)
    {
      std::cout<<matriz_C_paralela[i][c]<< ", ";
    }
    std::cout<<std::endl;
  }
  std::cout<<"]"<<std::endl<<std::endl; 

  std::cout<<" matriz secuencial :"<<std::endl;
  std::cout<<"["; 
  for(int i=0;i<cantidad_N;i++)
  {
    for(int c=0;c<cantidad_N;c++)
    {
      std::cout<<matriz_C_secuencial[i][c]<< ", ";
    }
    std::cout<<std::endl;
  }
  std::cout<<"]"<<std::endl; 
  */

  TIEMPO_FIN( HTH_TOTAL )

 std::cout<<cantidad_N/2<<std::endl;
 std::cout<<"EJERCICIO DE SUMA DE MATRICES"<<std::endl;
 std::cout<<"C = alfa*A + beta*B"<<std::endl;
 std::cout<<"se realizo "<<cantidad_N*cantidad_N<<" operaciones de suma"<<std::endl;
 std::cout<<"Valor ALFA       : "<<alfa<<std::endl;
 std::cout<<"Valor BETA       : "<<beta<<std::endl;
 std::cout<<"MATRIZ CUADRADA  : "<<cantidad_N<<std::endl; 
 std::cout<<"Valores Reales   : "<<std::endl;
 std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;
 std::cout<<std::endl;
 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


 std::cout<<std::endl;
 std::cout<<"Valores Ideal: "<<std::endl;
 TIEMPO_GET(HTH_AXPY_OMP) = TIEMPO_GET(HTH_AXPY_SEC) / 2;
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;

 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


}
// ----------------------------------------------------------------------------

"""
text_file = open("suma_matriz_axpy.cpp", "w")
text_file.write(code)
text_file.close()

## 2.1 Compila el codigo de C++.

In [ ]:
!g++ -o suma_matriz -fopenmp suma_matriz_axpy.cpp

# 3 Desarrollo
Ejecución del programa que realiza la suma de matrices


In [ ]:
try:
  %env OMP_NUM_THREADS=2
  !./suma_matriz 2 5 18000
except Exception as e:
  print("Oops Ocurrio una error!")
  print("Error debido a: ", e.__class__)
  print(e)

env: OMP_NUM_THREADS=2
9000
EJERCICIO DE SUMA DE MATRICES
C = alfa*A + beta*B
se realizo 324000000 operaciones de suma
Valor ALFA       : 2
Valor BETA       : 5
MATRIZ CUADRADA  : 18000
Valores Reales   : 
Tiempo TOTAL     : 23000.3 [ms]
Tiempo axpy Sec  : 7058.57 [ms]
Tiempo axpy Omp  : 5295.55 [ms]

SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 7058.57 / 5295.55 = 1.33292
Eficiencia       : SpeedUp/nro procesadores            : 1.33292 / 2 = 0.666462
Coste Sec        : nro procesadores*Tiempo             : 1 * 7058.57 = 7058.57
Coste Omp        : nro procesadores*Tiempo             : 2 * 5295.55 = 10591.1
Funcion Overhead : Coste Omp - tiempo Secuencial       : 10591.1 - 7058.57 = 3532.53

Valores Ideal: 
Tiempo axpy Sec  : 7058.57 [ms]
Tiempo axpy Omp  : 3529.29 [ms]
SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 7058.57 / 3529.29 = 2
Eficiencia       : SpeedUp/nro procesadores            : 2 / 2 = 1
Coste Sec        : nro procesadores*Tiempo             : 1 * 7

---
# 4 Tabla de pasos


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      | pip install pycuda     | Instala en el cuaderno los driver de CUDA para Python.
CPU      |  import                | Importa los módulos.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |numpy.random.randn(n, m)| Crear una matrices de dimensiones nxm con numeros ramdoms.
CPU      |  ar.astype(np.int32()) | Convierte los elemementos del array ar en elementos de tipo int 32 bit.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria para las matrices en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copio los valores en crudo de las matrices al GPU.
CPU      |  SourceModule()        | Posee el còdigo del kernel.
CPU      |  module.get_function() | convierte el texto del kernel en funcion de Python.
CPU      | dim_hilo_x, dim_hilo_y | Calcula las dimensiones para la ejecuciòn de 2D.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU, enviando los parametros.
CPU      |  print()               | Informa con algun mensaje.
CPU      | cuda.memcpy_dtoh()     | Copia desde la memoria GPU al CPU.

#5 Conclusiones


---
# 6 Bibliografía

[1] Multiplicacion de matrices, conceptual: [Pagina economipedia](https://economipedia.com/definiciones/multiplicacion-de-matrices.html)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Mulplicacion de matrices. Aplicaciones y Desarrollo: [PDF](http://docs.uprb.edu/deptmate/material%20suplementario/CIME/7mo%20a%209no/Matrices%20y%20sus%20Aplicaciones.pdf)

[4] Canal Derivando, Aplicaciones de Matrices: [Video](https://www.youtube.com/watch?v=9FKFgNQktkU)
